# 📊 Unión y Limpieza de datos del Dataset  

---

**Objetivo del Notebook**  
Limpieza de datos, columnas innecesarias y valores nulos/blancos 

**Contexto del análisis**  
- Dataset de muestra proporcionado + csv proporcionado unido en un único excel dataset
- Enfoque en aprendizaje, validación del pipeline y comprensión del proceso

**Valor devuelto**  
- Copia del Dataset de muestra proporcionado completamente limpio y útil 

---




In [ ]:
import pandas as pd
import xlsxwriter
import utils
# ===============================
# LEER EL ARCHIVO LIMPIO
# ===============================
ruta = r"C:\Users\0017655\Downloads\DataSET_SF - V2.xlsx"
dfs = pd.read_excel(ruta, sheet_name=None)

# Ver la primera hoja
oportunidad = list(dfs.values())[0]
cuenta = list(dfs.values())[1]
ecb = list(dfs.values())[2]
solicitud_ban = list(dfs.values())[3]
casos = list(dfs.values())[4]
correos = list(dfs.values())[5]
historial_actividad = list(dfs.values())[6]
historial_etapas = list(dfs.values())[7]


print(historial_etapas.head())

In [ ]:
from utils import analisis_na_por_columna
analisis_na_por_columna(oportunidad)
analisis_na_por_columna(cuenta)
analisis_na_por_columna(ecb)
analisis_na_por_columna(solicitud_ban)
analisis_na_por_columna(casos)
analisis_na_por_columna(correos)
analisis_na_por_columna(historial_actividad)
analisis_na_por_columna(historial_etapas)

In [ ]:
from utils import analisis_na_por_columna, eliminar_columnas_na


# ===============================
# LIMPIEZA DE NAS
# ===============================

oportunidad = eliminar_columnas_na(oportunidad)
cuenta = eliminar_columnas_na(cuenta)
ecb = eliminar_columnas_na(ecb)
solicitud_ban = eliminar_columnas_na(solicitud_ban)
casos = eliminar_columnas_na(casos)
correos = eliminar_columnas_na(correos)
historial_actvidad = eliminar_columnas_na(historial_actividad)
historial_etapas = eliminar_columnas_na(historial_etapas)

# ===============================
# CREACION DEL TARJET
# ===============================
def crear_target(oportunidad: pd.DataFrame, historial_etapas: pd.DataFrame) -> pd.DataFrame:
    """
    Crea la columna 'target' para cada oportunidad según la lógica:
    - Existe etapa 'Matrícula OOGG' con estado 'formalizada'
    - No existe etapa 'Desmatriculado'
    """
    
    # Filtrar historial por Matrícula OOGG y estado formalizada
    matricula_formalizada = historial_etapas[
        (historial_etapas['PL_Etapa__c'] == 'Matrícula OOGG') &
        (historial_etapas['PL_Subetapa__c'] == 'Formalizada')
    ]['LK_Oportunidad__c'].unique()
    print('Hay un total de '+str(len(matricula_formalizada))+' matrículas formalizadas. Un '+str(round(len(matricula_formalizada)/len(historial_etapas['LK_Oportunidad__c'].unique())*100,2))+'% del total de oportunidades')

    
    
    # Filtrar historial por Desmatriculado
    desmatriculado = historial_etapas[
        historial_etapas['PL_Subetapa__c'] == 'Desmatriculado'
    ]['LK_Oportunidad__c'].unique()
    print('Hay un total de '+str(len(desmatriculado))+' desmatriculados. Un '+str(round(len(desmatriculado)/len(matricula_formalizada)*100,2))+'% del total de matriculados')
    # Crear target: 1 si está en matricula formalizada y no en desmatriculado
    oportunidad['target'] = oportunidad['ID'].apply(
        lambda x: 1 if (x in matricula_formalizada and x not in desmatriculado) else 0
    )
    # Crear desmatriculado: 1 si está Desmatriculado y 0 en caso contrario
    oportunidad['desmatriculado'] = oportunidad['ID'].apply(
        lambda x: 1 if (x in desmatriculado) else 0
    )
    
    return oportunidad

target = crear_target(oportunidad, historial_etapas)
target.columns




# Análisis descriptivo (Seguimiento 1)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Resumen numérico con cálculo de porcentaje
resumen_acceso = target.groupby(['PL_CURSO_ACADEMICO', 'PL_ORIGEN_DE_SOLICITUD', 'target'])['ID'].nunique().unstack(fill_value=0)
resumen_acceso.columns = ['No Matriculado (0)', 'Matriculado (1)']

# Calcular Total y % de Matriculados (Tasa de Conversión/Fidelidad)
resumen_acceso['Total'] = resumen_acceso['No Matriculado (0)'] + resumen_acceso['Matriculado (1)']
resumen_acceso['% Fidelidad'] = (resumen_acceso['Matriculado (1)'] / resumen_acceso['Total'] * 100).round(2)


# Gráfico Global de Acceso
plt.figure(figsize=(10, 6))
sns.countplot(data=target.drop_duplicates('ID'), x='PL_ORIGEN_DE_SOLICITUD', hue='target', palette='viridis')
plt.title('Volumen de Oportunidades por Tipo de Acceso')
plt.xticks(rotation=45)
plt.show()
resumen_acceso

In [ ]:
from utils import graficar_top_por_acceso
import matplotlib.pyplot as plt
# Ejecución
graficar_top_por_acceso(target, top_n=5)

In [ ]:
# 1. Corregimos la lista (quitamos duplicados y añadimos comas faltantes)
columnas_seleccionadas = [
    'ACCOUNTID', 'ID', 'ID18__PC', 'target', 'desmatriculado', 'PL_CURSO_ACADEMICO', 'CH_NACIONAL',
    'NU_NOTA_MEDIA_ADMISION', 'NU_NOTA_MEDIA_1_BACH__PC', 'CH_PRUEBAS_CALIFICADAS', 
    'NU_RESULTADO_ADMISION_PUNTOS', 'PL_RESOLUCION_DEFINITIVA', 'TITULACION', 'CENTROENSENANZA',
    'MINIMUMPAYMENTPAYED', 'PAID_AMOUNT', 'PAID_PERCENT', 'CH_PAGO_SUPERIOR', 
    'CH_MATRICULA_SUJETA_BECA', 'CH_AYUDA_FINANCIACION', 'CU_IMPORTE_TOTAL',
    'CH_VISITACAMPUS__PC', 'CH_ENTREVISTA_PERSONAL__PC', 'ACC_DTT_FECHAULTIMAACTIVIDAD', 
    'NU_PREFERENCIA', 'STAGENAME', 'PL_SUBETAPA',
    'CH_HIJO_EMPLEADO__PC', 'CH_HIJO_PROFESOR_ASOCIADO__PC', 'CH_HERMANOS_ESTUDIANDO_UNAV__P', 
    'CH_HIJO_MEDICO__PC', 'YEARPERSONBIRTHDATE', 'NAMEX', 'CH_FAMILIA_NUMEROSA__PC', 
    'PL_SITUACION_SOCIO_ECONOMICA', 'LEADSOURCE', 'PL_ORIGEN_DE_SOLICITUD', 
    'PL_PLAZO_ADMISION', 'RECORDTYPENAME'
]

# 2. Unión asegurando que no arrastramos basura
df_unido = pd.merge(
    target, 
    cuenta, 
    left_on='ACCOUNTID', 
    right_on='ID18', 
    how='left',
    suffixes=('', '_cuenta')
)

# 3. Filtrado y ELIMINACIÓN DE COLUMNAS DUPLICADAS (por si acaso)
columnas_finales = []
for col in columnas_seleccionadas:
    if col in df_unido.columns and col not in columnas_finales:
        columnas_finales.append(col)

df_unido_filtrado = df_unido[columnas_finales].copy()

# 4. TRUCO FINAL: Limpiar el índice para que la función no explote
df_unido_filtrado = df_unido_filtrado.drop_duplicates(subset=['ID']).reset_index(drop=True)


In [ ]:
from utils import calcular_tiempos_etapas
import numpy as np
import pandas as pd

historial_etapas_tiempo = calcular_tiempos_etapas(historial_etapas)

def limpiar_historial_por_hitos(df_historial, df_principal):
    # 1. Asegurar formato datetime y TRABAJAR SOBRE UNA COPIA
    df_h = df_historial.copy()
    df_h['CreatedDate'] = pd.to_datetime(df_h['CreatedDate'])
    
    # 2. Hitos (Igual que antes pero asegurando que no haya basura en el index)
    hito_acad = df_h[
        (df_h['PL_Etapa__c'] == 'Pruebas de admisión') & 
        (df_h['PL_Subetapa__c'] == 'Pruebas calificadas')
    ].groupby('LK_Oportunidad__c')['CreatedDate'].min().reset_index()
    hito_acad.columns = ['LK_Oportunidad__c', 'fecha_pruebas_calificadas']
    
    hito_econ = df_h[
        (df_h['PL_Etapa__c'] == 'Matrícula admisión') & 
        (df_h['PL_Subetapa__c'] == 'Pago Mínimo')
    ].groupby('LK_Oportunidad__c')['CreatedDate'].min().reset_index()
    hito_econ.columns = ['LK_Oportunidad__c', 'fecha_matricula_iniciada']
    
    # 3. Unir hitos
    df_merge = pd.merge(df_h, hito_acad, on='LK_Oportunidad__c', how='left')
    df_merge = pd.merge(df_merge, hito_econ, on='LK_Oportunidad__c', how='left')
    
    # 4. PREVENCIÓN: Antes del merge final, eliminar del df_principal 
    # las columnas que YA están en df_merge para evitar duplicados (.x, .y)
    cols_a_quitar = [c for c in df_principal.columns if c in df_merge.columns and c != 'ID']
    df_principal_clean = df_principal.drop(columns=cols_a_quitar)
    
    # 5. Merge final
    df_final = pd.merge(df_merge, df_principal_clean, left_on='LK_Oportunidad__c', right_on='ID', how='left')
    
    # 6. Definición de grupos (Asegúrate de que NO hay duplicados aquí)
    cols_academicas = list(set([
        'NU_NOTA_MEDIA_ADMISION', 'CH_PRUEBAS_CALIFICADAS', 
        'NU_RESULTADO_ADMISION_PUNTOS', 'PL_RESOLUCION_DEFINITIVA'
    ]))
    
    cols_economicas = list(set([
        'MINIMUMPAYMENTPAYED', 'PAID_AMOUNT', 'PAID_PERCENT', 'CH_PAGO_SUPERIOR', 
        'CH_MATRICULA_SUJETA_BECA', 'CH_AYUDA_FINANCIACION', 'CU_IMPORTE_TOTAL'
    ]))
    
    # 7. Reset index para evitar errores de reindexación
    df_final = df_final.reset_index(drop=True)
    
    # 8. Aplicación de la lógica temporal
    mask_acad = (df_final['fecha_pruebas_calificadas'].isna()) | (df_final['CreatedDate'] < df_final['fecha_pruebas_calificadas'])
    df_final.loc[mask_acad, cols_academicas] = np.nan
    
    mask_econ = (df_final['fecha_matricula_iniciada'].isna()) | (df_final['CreatedDate'] < df_final['fecha_matricula_iniciada'])
    df_final.loc[mask_econ, cols_economicas] = np.nan
    
    return df_final

df_final = limpiar_historial_por_hitos(historial_etapas_tiempo, df_unido_filtrado)

ejemplo_id = '0066900001k7yTgAAI'

columnas_comprobacion = [
    'LK_Oportunidad__c', 'CreatedDate', 'PL_Etapa__c', 'PL_Subetapa__c',
    'fecha_pruebas_calificadas', 'NU_NOTA_MEDIA_ADMISION',
    'fecha_matricula_iniciada', 'PAID_AMOUNT', 'CH_PRUEBAS_CALIFICADAS', 
        'NU_RESULTADO_ADMISION_PUNTOS', 'PL_RESOLUCION_DEFINITIVA'
]

print("--- COMPROBACIÓN DE LÓGICA TEMPORAL ---")
df_final[df_final['LK_Oportunidad__c'] == ejemplo_id][columnas_comprobacion].sort_values('CreatedDate')
#df_final.head(50)

In [ ]:
import pandas as pd
import numpy as np
from utils import integrar_actividades_progresivo_por_curso
# Ejecución
df_final_v3 = integrar_actividades_progresivo_por_curso(df_final, historial_actividad)
# ==========================================
# EJECUCIÓN
# ==========================================
# Print de comprobación para ver la evolución de un contacto
ejemplo_acc = df_final_v3[df_final_v3['num_asistencias_acum'] > 0]['ID18__PC'].iloc[1]
cols_print = ['ID','ID18__PC','ACCOUNTID', 'CreatedDate', 'PL_Etapa__c', 'num_asistencias_acum', 'num_solicitudes_acum']

print("\n--- COMPROBACIÓN DE EVOLUCIÓN DE ACTIVIDADES ---")
df_final_v3[df_final_v3['ID18__PC'] == ejemplo_acc][cols_print].sort_values('CreatedDate')
#historial_actividad.loc[historial_actividad['ContactId']=='003690000312P6pAAE',]
df_final_v3.head(100)
df_integrado = df_final_v3

In [ ]:
# ===============================
# 3. GUARDAR EXCEL LIMPIO
# ===============================
#df_integrado.to_csv(r"..\datos\02. Datos tratamiento preliminar\01_datos_tratamiento_preliminar - V2.csv",sep=";")

print("Archivo limpio guardado como '01_datos_tratamiento_preliminar - V2.csv'")

# 📊 Creación del Dataset Maestro

---

**Objetivo del Notebook**  
Creacion del dataset maestro una vez obtenido el dataset limpio

**Contexto del análisis**  
- Dataset limpio en etapas anteriores

**Valor devuelto**  
- Copia del Dataset de muestra proporcionado completamente funcional

---




TAREA 1: Comprobar que el conjunto de datos contiene todas las variables importantes de la pestaña de cuenta: Variables relacionadas con el origen del alumno: nacional o internacional, colegio de procedencia, si el colegio es afín o no a la UNAV, si es hijo de empleado o de profesor asociado (por los descuentos en matrícula), si es familia numerosa y de qué tipo.

In [ ]:
# ===============================
# VARIABLES CLAVE ESPERADAS (CUENTA)
# ===============================

variables_cuenta_clave = {
    "Origen del alumno": [
        "PL_NACIONALIDAD__C",          # cambiar por colegio
        "PL_TIPO_ALUMNO__C"            # Nacional / Internacional (alternativa frecuente)
    ],
    
    "Colegio de procedencia": [
        "CENTROENSENANZA",
        "CH_CENTRO_AFIN_UNAV__C"       # Colegio afín a UNAV, añadir nota media 1º bachillerato, hijo empleado, hijo medico, hijo hermanos en la unav, etc
    ],
    
    "Vínculo con UNAV (descuentos)": [
        "CH_HIJO_EMPLEADO__PC",
        "CH_HIJO_PROFESOR_ASOCIADO__C" # si existe
    ],
    
    "Familia numerosa": [
        "CH_FAMILIA_NUMEROSA__PC",
        "PL_TIPO_FAMILIA_NUMEROSA__C"  # general / especial, añadir nº miembros de la familia, 
    ]
}



In [ ]:
# ===============================
# COMPROBACIÓN DE EXISTENCIA EN CUENTA: verificamos qué está y qué falta realmente en el dataframe cuenta.
# ===============================

columnas_cuenta = set(cuenta.columns)

estado_variables = []

for bloque, vars_bloque in variables_cuenta_clave.items():
    for var in vars_bloque:
        estado_variables.append({
            "Bloque": bloque,
            "Variable": var,
            "Existe_en_cuenta": var in columnas_cuenta
        })

df_estado_variables = pd.DataFrame(estado_variables)
df_estado_variables


In [ ]:
# ===============================
# COMPROBAR PRESENCIA EN DATASET FINAL: comprobamos si esas variables han sobrevivido al cruce y están en el dataset final.
# ===============================

columnas_final = set(df_integrado.columns)

df_estado_variables["Existe_en_df_integrado"] = (
    df_estado_variables["Variable"].isin(columnas_final)
)

df_estado_variables


In [ ]:
# ===============================
# CALIDAD DE VARIABLES EXISTENTES: Para las que sí existen en df_integrado, comprobamos si tienen datos útiles.
# ===============================

vars_validas = df_estado_variables.loc[
    df_estado_variables["Existe_en_df_integrado"],
    "Variable"
].tolist()

calidad = (
    df_integrado[vars_validas]
    .isna()
    .mean()
    .reset_index()
    .rename(columns={"index": "Variable", 0: "% NA"})
)

calidad["% NA"] = (calidad["% NA"] * 100).round(2)
calidad.sort_values("% NA", ascending=False)


In [ ]:
# ===============================
# CONCLUSIÓN AUTOMÁTICA
# ===============================

conclusion = df_estado_variables.copy()
conclusion["Estado"] = np.select(
    [
        ~conclusion["Existe_en_cuenta"],
        conclusion["Existe_en_cuenta"] & ~conclusion["Existe_en_df_integrado"],
        conclusion["Existe_en_df_integrado"]
    ],
    [
        "❌ No existe en origen",
        "⚠️ Existe en cuenta pero no llegó al dataset final",
        "✅ Disponible en dataset final"
    ]
)

# Mostrar la conclusión
display(conclusion)

# ===============================
# REVISAR NACIONALIDAD / VARIABLES DIFERENTES
# ===============================

# Filtrar las variables que no existen en el origen
vars_no_origen = conclusion.loc[conclusion["Estado"] == "❌ No existe en origen", "Variable"].tolist()

if vars_no_origen:
    print("\nVariables que no existen en el origen según la base de datos:")
    print(vars_no_origen)

    # Revisar si estos nombres aparecen en el Excel con otro nombre
    posibles_renombradas = []
    for var in vars_no_origen:
        # Buscar columnas que contengan partes del nombre
        matches = [c for c in cuenta.columns if var.lower() in c.lower() or c.lower() in var.lower()]
        if matches:
            posibles_renombradas.append((var, matches))
    
    if posibles_renombradas:
        print("\nPosibles coincidencias en el Excel (renombradas o distintas):")
        for original, encontrados in posibles_renombradas:
            print(f"{original} -> {encontrados}")
    else:
        print("\n⚠️ No se encontraron coincidencias en el Excel, estas variables habría que solicitarlas.")


#REVISAR LA NACIONALIDAD POR SI SE LLAMA DISTINTO (EN EL EXCEL)
# Juan: aquí hay que comprobar en el Excel si los valores que no exixsten en el origen están en el Excel y no aparecen porque se llama de otra forma o si es porque directamente no aparecen y habría que solicitarlos


TAREA 2: Comprobar que el target se ha creado correctamente, que no hay valores vacíos. En caso de tener valores vacíos, comprobar de donde vienen y porqué ocurren. Comprobar que las variables importantes de oportunidad aparecen en el conjunto de datos: tipo de solicitud (Información o admisión), plazo de admisión(con un tratamiento de si es Diciembre, Marzo o Rolling [que es cuando solicitas la prueba y la haces a los días, suele aparecer en blanco] y tratar de construir bien esta variable), nu_preferencia (con el orden de preferencia), si ha pagado, el pago mínimo, notas de admisión, etc. Asegurar que no aparece información de futuro en etapas anteriores. 


In [ ]:
# ===============================
# ELIMINAR REGISTROS CON TARGET NULO
# ===============================

# Contar antes
num_nulos = df_integrado['target'].isna().sum()
print(f"Eliminando {num_nulos} registros con target nulo...")

# Filtrar el dataframe
df_integrado = df_integrado[df_integrado['target'].notna()].copy()

# Comprobación rápida
print("Número de registros tras eliminar nulos:", len(df_integrado))
print("Valores únicos del target ahora:")
print(df_integrado['target'].value_counts())

# Juan: El target que sea nulo hay que eliminarlo, ya que no se puede modelar con target nulo. Este caso es seguro uno de los casos sensibles eliminados.

In [ ]:
# ===============================
# VARIABLES CLAVE DE OPORTUNIDAD
# ===============================

variables_op_clave = {
    "Tipo de solicitud": [
        "PL_ORIGEN_DE_SOLICITUD",  # Información / Admisión
        "CH_ORIGEN_ADMISION",
        "RECORDTYPENAME"
    ],
    
    "Plazo de admisión": [
        "PL_PLAZO_ADMISION"
    ],
    
    "Preferencia": [
        "NU_PREFERENCIA"
    ],
    
    "Pago": [
        "CH_PAGADO__C",
        "MINIMUMPAYMENTPAYED",
        "IMPORTE_MINIMO_PERSONALIZADO"
    ],
    
    "Notas admisión": [
        "NU_NOTA_MEDIA_ADMISION",
        "NU_RESULTADO_ADMISION_PUNTOS"
    ]
}

# ===============================
# EXISTENCIA DE VARIABLES
# ===============================

estado_op = []

for bloque, vars_bloque in variables_op_clave.items():
    for var in vars_bloque:
        estado_op.append({
            "Bloque": bloque,
            "Variable": var,
            "Existe_en_df_integrado": var in df_integrado.columns
        })

df_estado_op = pd.DataFrame(estado_op)
df_estado_op


In [ ]:
import numpy as np

df_integrado['PL_TIPOSOLICITUD'] = np.select([df_integrado['RECORDTYPENAME'].str.contains('grado', case=False, na=False), df_integrado['RECORDTYPENAME'].str.contains('m[áa]ster', case=False, na=False)], ['Grado', 'Máster'], default='Otro')

In [ ]:
# ===============================
# CONSTRUCCIÓN PLAZO LIMPIO 
# ===============================

def normalizar_plazo(row):
    plazo = row['PL_PLAZO_ADMISION']
    tipo_carrera = row['PL_TIPOSOLICITUD']  # Asegúrate de tener esta columna (Grado / Máster)
    
    # Master siempre Rolling
    if tipo_carrera.lower() == "master":
        return "Rolling"
    
    # Nulos → Rolling
    if pd.isna(plazo):
        return "Rolling"
    
    # Normalización de otros plazos
    plazo = str(plazo).strip().lower()
    if "dic" in plazo:
        return "Diciembre"
    if "mar" in plazo:
        return "Marzo"
    
    # Todo lo demás → Otros
    return "Otros"

# Aplicamos
df_integrado['PLAZO_ADMISION_LIMPIO'] = df_integrado.apply(normalizar_plazo, axis=1)

# ===============================
# ANÁLISIS DE DISTRIBUCIÓN
# ===============================

# Por Plazo en bruto
print("Distribución Plazo en bruto:")
print(df_integrado['PL_PLAZO_ADMISION'].value_counts(dropna=False))

# Por Plazo limpio
print("\nDistribución Plazo limpio:")
print(df_integrado['PLAZO_ADMISION_LIMPIO'].value_counts())

# Por Plazo limpio y tipo de carrera
print("\nDistribución Plazo limpio por tipo de carrera:")
display(df_integrado.groupby(['TITULACION','PLAZO_ADMISION_LIMPIO']).size().reset_index(name='count'))


#Juan: Aquí me surge la duda, ya que ellos dijeron que las pruebas son Diciembre, Marzo y Rolling, pero cuando vemos el Plazo de admisión, hay en casi todos los meses, ¿entra en la de diciembre la de noviembre? ¿Y en la de marzo entra febrero y abril? o que nos digan cómo podemos detectar cuando es Rolling o no.

# Juan: Master ponerlo siempre a Rolling, y el resto dejarlo con la lógica de Diciembre, Marzo y Rolling. Después hacer un estudio valuecounts por Plazo en bruto, plazo limpio y tipo de carrera: grado o máster

In [ ]:
df_integrado.columns.tolist

In [ ]:
# ===============================
# COMPROBACIÓN VARIABLES ECONÓMICAS Y DE PAGO
# ===============================

# Juan: Variables a NaN
# Variables económicas que queremos analizar
vars_pago_esperadas = [
    'CU_IMPORTE_TOTAL',
    'PAID_AMOUNT',
    'PAID_PERCENT'
]

# Filtrar solo las columnas que existen en df_integrado
vars_pago_existentes = [v for v in vars_pago_esperadas if v in df_integrado.columns]

# Mostrar cuáles existen y cuáles no
print("Variables encontradas en df_integrado:", vars_pago_existentes)
vars_faltantes = [v for v in vars_pago_esperadas if v not in df_integrado.columns]
if vars_faltantes:
    print("⚠️ Variables no encontradas en df_integrado (no se incluyen en análisis):", vars_faltantes)

# ===============================
# NORMALIZAR VARIABLES DE PAGO
# ===============================

# Convertir a numérico las columnas de pago originales (forzando errores a NaN)

# ===============================
# DESCRIPCIÓN ESTADÍSTICA
# ===============================

print("\n--- Descripción de las variables de pago existentes ---")
display(df_integrado[['CU_IMPORTE_TOTAL', 'PAID_AMOUNT', 'PAID_PERCENT']].describe())

# ===============================
# DETECCIÓN DE INFORMACIÓN FUTURA
# ===============================

if 'PAID_AMOUNT' in df_integrado.columns and 'PL_Etapa__c' in df_integrado.columns:
    casos_incoherentes = df_integrado[
        (df_integrado['PL_Etapa__c'] != 'Matrícula') & 
        (df_integrado['PAID_AMOUNT'] > 0)
    ]
    print(f"\n⚠️ Casos con pagos antes de Matrícula: {len(casos_incoherentes)}")
    display(casos_incoherentes[['ID', 'PL_Etapa__c', 'PAID_AMOUNT', 'target']])

# Revisar registros con pagos positivos
pagos_positivos = df_integrado[df_integrado['PAID_AMOUNT'] > 0]
print(f"\nRegistros con PAID_AMOUNT > 0: {len(pagos_positivos)}")
display(pagos_positivos[['ID','PAID_AMOUNT','PAID_PERCENT','MINIMUMPAYMENTPAYED']])

In [ ]:


df_integrado['NU_PREFERENCIA'].describe()
#df_integrado[['CH_PAGADO__C', 'MINIMUMPAYMENTPAYED', 'IMPORTE_MINIMO_PERSONALIZADO']].describe()


# ===============================
# DETECCIÓN DE INFORMACIÓN FUTURA
# ===============================

casos_incoherentes = df_integrado[
    (df_integrado['PL_Etapa__c'] != 'Matrícula') &
    (df_integrado['PAID_AMOUNT'] > 0)
]

print("Casos con pago antes de etapa de matrícula:",
      len(casos_incoherentes))

df_final_filtrado = df_integrado.copy()

TAREA 3: De la pestaña ECB nos interesan tres variables por oportunidad: La renta familiar, el coste ordinario (el coste sin aplicar ningún tipo de descuento) y el Importe matrícula a pagar (que es el valor final que es el importe que paga el alumno con todos los descuentos aplicados), el % del total que acaba pagando el alumno (importe matrícula/coste ordinario * 100). Importante que no aparezca esta variable informada antes de que se realice el estudio de la beca, se puede comprobar con la fecha de la etapa en la que se encuentra la oportunidad y la fecha de solicitud de la beca, en el caso que no haya un cruce más sencillo)

In [ ]:
df_integrado = df_final_filtrado.merge(
    ecb[
        [
            'LK_oportunidad__c',
            'FO_rentaMEC_for__c',
            'FO_rentaFam_ges__c',
            'CU_precioOrdinario_def__c',
            'CU_precioIncentivado_def__c',
            'CU_precioFamNum_def__c',
            'PO_descFamNum_def__c',
            'CU_precioAplicado_def__c'
        ]
    ],
    left_on='ID',
    right_on='LK_oportunidad__c',
    how='left'
)

# ===============================
# VARIABLES ECONÓMICAS ECB FINALES
# ===============================

vars_ecb = [
    'FO_rentaFam_ges__c',
    'CU_precioOrdinario_def__c',
    'CU_precioAplicado_def__c'
]

print("Variables ECB disponibles:")
print([v for v in vars_ecb if v in df_integrado.columns])

df_integrado[vars_ecb].describe()


In [ ]:
# ===============================
# % PAGADO SOBRE COSTE ORDINARIO
# ===============================

# Calcular porcentaje pagado
df_integrado['PORCENTAJE_PAGADO_FINAL'] = (
    df_integrado['CU_precioAplicado_def__c'] /
    df_integrado['CU_precioOrdinario_def__c']
) * 100

# Control de valores no válidos (división por cero, negativos o >100)
df_integrado.loc[
    (df_integrado['CU_precioOrdinario_def__c'] <= 0) |
    (df_integrado['PORCENTAJE_PAGADO_FINAL'] < 0) |
    (df_integrado['PORCENTAJE_PAGADO_FINAL'] > 100),
    'PORCENTAJE_PAGADO_FINAL'
] = np.nan

# Estadísticas y comprobación
print("--- Estadísticas PORCENTAJE_PAGADO_FINAL ---")
display(df_integrado['PORCENTAJE_PAGADO_FINAL'].describe())

# Mostrar registros fuera de rango (solo si hay)
fuera_rango = df_integrado[
    (df_integrado['PORCENTAJE_PAGADO_FINAL'] < 0) |
    (df_integrado['PORCENTAJE_PAGADO_FINAL'] > 100)
]
if len(fuera_rango) > 0:
    print(f"\n⚠️ Registros fuera de 0-100: {len(fuera_rango)}")
    display(fuera_rango[['ID', 'CU_precioAplicado_def__c', 'CU_precioOrdinario_def__c', 'PORCENTAJE_PAGADO_FINAL']])
else:
    print("\n✅ Todos los valores de PORCENTAJE_PAGADO_FINAL están entre 0 y 100 o son NaN.")

#Juan: Comprobar que el porcentaje pagado final entra dentro de 0-100

In [ ]:
# Juan: Me parece más robusto ordenar por fecha de creación de las etapas, detectar cual es la primera en la que se paga, y en estapas anteriores a esa comprobar que siempre son NA, mirar construcción de limpiar_por_historial en Utils

# ===============================
# DETECCIÓN DE INFORMACIÓN FUTURA (BECAS / ECB)
# ===============================

# Ordenamos las etapas por fecha de creación (asumiendo que existe 'fecha_creacion_etapa')
df_integrado = df_integrado.sort_values(['ID', 'fecha_creacion_etapa'])

# Detectar la primera etapa en la que hay pago aplicado
df_integrado['primer_pago'] = df_integrado.groupby('ID')['CU_precioAplicado_def__c'].transform('first')

# Casos donde hay pago antes de la primera etapa con pago
casos_info_futura = df_integrado[
    (df_integrado['CU_precioAplicado_def__c'].notna()) &
    (df_integrado['CU_precioAplicado_def__c'] != df_integrado['primer_pago'])
]

print("⚠️ Casos con importe final antes de la primera etapa de pago:", len(casos_info_futura))

display(casos_info_futura[['ID', 'PL_Etapa__c', 'CU_precioAplicado_def__c', 'FO_rentaFam_ges__c']])


In [ ]:
df_integrado.loc[
    df_integrado['PL_Etapa__c'].isin(etapas_previas_beca),
    ['CU_precioAplicado_def__c', 'PORCENTAJE_PAGADO_FINAL']
] = np.nan


TAREA 4: De la pestaña de etapas, obtener el tiempo que lleva en cada etapa. En caso de ser la etapa actual, que se calcule como el tiempo entre el inicio de la etapa y la fecha de hoy en días.

In [ ]:
import importlib
import utils

importlib.reload(utils)


In [ ]:
from utils import calcular_tiempos_etapas
# ===============================
# TIEMPO EN CADA ETAPA (USANDO UTILS)
# ===============================

df_etapas = calcular_tiempos_etapas(historial_etapas)
df_etapas.head()


# Juan: Hay una función en utils que lo hace: calcular_tiempo_etapas

TAREA 5: De la pestaña de historial de actividades obtener el número de actividades que lleva asistidas hasta esa etapa, comprobar que se calcula bien. Si da tiempo, añadir las actividades de la pestaña casos que son "Asistencia familias" para que se cuente como actividad. Importante comprobar que no se cuentan actividades futuras.

In [ ]:
# ===============================
# LIMPIEZA ACTIVIDADES
# ===============================
# Juan: Hay una función en utils. Hay que añadir las actividades de tipo "Familiar" de la pestaña "Casos"
df_act = df_actividades.copy()

# Fecha actividad
df_act['fecha_actividad'] = pd.to_datetime(
    df_act['ActivityDate'],
    errors='coerce'
)

# Nos quedamos solo con actividades asistidas / completadas
df_act = df_act[
    df_act['Status'].isin(['Asistida', 'Completada', 'Completed'])
]

df_act = df_act[
    ['LK_Oportunidad__c', 'fecha_actividad']
].dropna()

#Restore hasta aqui

TAREA 6: Comprobar varios ejemplos y asegurar que no hay variables informadas con información del futuro.

In [ ]:
# ============================================================
# TAREA 6 · COMPROBACIÓN DE INFORMACIÓN DEL FUTURO (LEAKAGE)
# ============================================================

import pandas as pd
import numpy as np

print("🔍 INICIO COMPROBACIÓN DE INFORMACIÓN DEL FUTURO\n")

# ===============================
# 1. DEFINICIÓN DE REGLAS TEMPORALES
# ===============================

reglas_futuro = {
    "Pago y matrícula": {
        "etapas_no_permitidas": [
            'Solicitud',
            'Pruebas',
            'Admisión académica'
        ],
        "variables": [
            'PAID_AMOUNT',
            'MINIMUMPAYMENTPAYED',
            'CH_PAGADO__C',
            'CU_precioAplicado_def__c',
            'PORCENTAJE_PAGADO_FINAL'
        ]
    },
    
    "Resultados finales": {
        "etapas_no_permitidas": [
            'Solicitud',
            'Pruebas'
        ],
        "variables": [
            'PL_RESOLUCION_DEFINITIVA'
        ]
    }
}

# ===============================
# 2. DETECCIÓN AUTOMÁTICA
# ===============================

leakage_detectado = []

for bloque, regla in reglas_futuro.items():
    
    etapas = regla["etapas_no_permitidas"]
    variables = [v for v in regla["variables"] if v in df_final.columns]
    
    if not variables:
        continue
    
    mask = (
        df_final['PL_Etapa__c'].isin(etapas) &
        df_final[variables].notna().any(axis=1)
    )
    
    casos = df_final.loc[
        mask,
        ['ID', 'PL_Etapa__c'] + variables
    ].copy()
    
    if not casos.empty:
        casos['Bloque'] = bloque
        leakage_detectado.append(casos)

df_leakage = (
    pd.concat(leakage_detectado, ignore_index=True)
    if leakage_detectado
    else pd.DataFrame()
)

# ===============================
# 3. RESULTADOS GENERALES
# ===============================

if df_leakage.empty:
    print("✅ No se detecta información del futuro en el dataset.\n")
else:
    print(f"⚠️ Se detectan {df_leakage['ID'].nunique()} oportunidades con posible información futura.\n")
    display(df_leakage.head(10))

# ===============================
# 4. REVISIÓN MANUAL DE EJEMPLOS
# ===============================

if not df_leakage.empty:
    
    ejemplos_ids = df_leakage['ID'].unique()[:3]
    
    columnas_revision = [
        'ID',
        'CreatedDate',
        'PL_Etapa__c',
        'PAID_AMOUNT',
        'MINIMUMPAYMENTPAYED',
        'CU_precioAplicado_def__c',
        'PORCENTAJE_PAGADO_FINAL',
        'PL_RESOLUCION_DEFINITIVA',
        'target'
    ]
    
    columnas_revision = [
        c for c in columnas_revision if c in df_integrado.columns
    ]
    
    print("📌 REVISIÓN MANUAL DE EJEMPLOS:\n")
    
    display(
        df_integrado[
            df_integrado['ID'].isin(ejemplos_ids)
        ][columnas_revision]
        .sort_values(['ID', 'CreatedDate'])
    )

# ===============================
# 5. CONCLUSIÓN FINAL
# ===============================

if df_leakage.empty:
    print("🎯 CONCLUSIÓN: Dataset limpio temporalmente. Apto para modelado.")
else:
    print("🚨 CONCLUSIÓN: Revisar y corregir variables con información futura antes de modelar.")


In [ ]:
# ============================================================
# SCRIPT · DATASET DE TRATAMIENTO DEFINITIVO FINAL
# ============================================================

import pandas as pd
import numpy as np
from utils import crear_target, eliminar_columnas_na, calcular_tiempos_etapas, integrar_actividades_progresivo_por_curso
# ============================================================
# 1️⃣ CARGA DE DATOS
# ============================================================

ruta_excel = r'datos\01. Datos originales\DataSET_SF - V2.xlsx'
dfs = pd.read_excel(ruta_excel, sheet_name=None)

# Asignar cada hoja a un dataframe
oportunidad = list(dfs.values())[0]
cuenta = list(dfs.values())[1]
ecb = list(dfs.values())[2]
solicitud_ban = list(dfs.values())[3]
casos = list(dfs.values())[4]
correos = list(dfs.values())[5]
historial_actividad = list(dfs.values())[6]
historial_etapas = list(dfs.values())[7]

# ============================================================
# 2️⃣ LIMPIEZA INICIAL DE NAS Y COLUMNAS
# ============================================================

def eliminar_columnas_na(df, umbral=0.9):
    """Elimina columnas con más de un umbral de valores NA"""
    return df.loc[:, df.isna().mean() < umbral]

for df in [oportunidad, cuenta, ecb, solicitud_ban, casos, correos, historial_actividad, historial_etapas]:
    df = eliminar_columnas_na(df)

oportunidad = eliminar_columnas_na(oportunidad)
cuenta = eliminar_columnas_na(cuenta)
ecb = eliminar_columnas_na(ecb)

# ============================================================
# 3️⃣ CREACIÓN DEL TARGET
# ============================================================

oportunidad = crear_target(oportunidad, historial_etapas)

# Unión con cuenta

df_unido = pd.merge(
    oportunidad, 
    cuenta, 
    left_on='ACCOUNTID', 
    right_on='ID18', 
    how='left',
    suffixes=('', '_cuenta')
)


# ============================================================
# 4️⃣ CONSTRUCCIÓN VARIABLES DERIVADAS
# ============================================================

# Limpiar y crear plazo de admisión
def normalizar_plazo(x):
    if pd.isna(x): return "Rolling"
    x = str(x).strip().lower()
    if "dic" in x: return "Diciembre"
    if "mar" in x: return "Marzo"
    return "Otros"

df_unido['PLAZO_ADMISION_LIMPIO'] = df_unido['PL_PLAZO_ADMISION'].apply(normalizar_plazo)

# Variables económicas ECB
ecb_vars = ['LK_oportunidad__c', 'FO_rentaFam_ges__c', 'CU_precioOrdinario_def__c', 'CU_precioAplicado_def__c']
df_definitivo = pd.merge(
    df_unido,
    ecb[ecb_vars],
    left_on='ID',
    right_on='LK_oportunidad__c',
    how='left'
)

# % pagado
df_definitivo['PORCENTAJE_PAGADO_FINAL'] = (
    df_definitivo['CU_precioAplicado_def__c'] / df_definitivo['CU_precioOrdinario_def__c'] * 100
)
df_definitivo.loc[df_definitivo['CU_precioOrdinario_def__c'] <= 0, 'PORCENTAJE_PAGADO_FINAL'] = np.nan

ruta_salida = r"C:\Users\0017655\Downloads\dataset_analisis_final.csv"
df_definitivo.to_csv(ruta_salida, sep=";", index=False)

Hay un total de 15470 matrículas formalizadas. Un 22.03% del total de oportunidades
Hay un total de 1495 desmatriculados. Un 9.66% del total de matriculados


In [ ]:

from utils import calcular_tiempos_etapas, integrar_actividades_progresivo_por_curso

# ============================================================
# 5️⃣ TIEMPO EN CADA ETAPA
# ============================================================

historial_etapas_tiempo = calcular_tiempos_etapas(historial_etapas)
df_definitivo = historial_etapas_tiempo.merge(df_definitivo, left_on='LK_Oportunidad__c', right_on='ID', how='left')

# ============================================================
# 6️⃣ HISTORIAL DE ACTIVIDADES
# ============================================================

df_definitivo = integrar_actividades_progresivo_por_curso(df_definitivo, historial_actividad)

# ============================================================
# 7️⃣ CONTROL DE INFORMACIÓN FUTURA (LEAKAGE)
# ============================================================

etapas_pago = ['Solicitud', 'Pruebas', 'Admisión académica']
vars_pago = ['PAID_AMOUNT','MINIMUMPAYMENTPAYED','CU_precioAplicado_def__c','PORCENTAJE_PAGADO_FINAL']
vars_pago = [v for v in vars_pago if v in df_definitivo.columns]

mask_futuro = (df_definitivo['PL_Etapa__c'].isin(etapas_pago)) & (df_definitivo[vars_pago].notna().any(axis=1))
df_definitivo.loc[mask_futuro, vars_pago] = np.nan

# ============================================================
# 8️⃣ SELECCIÓN VARIABLES FINALES
# ============================================================
columnas_seleccionadas = [
    'ACCOUNTID', 'ID','ID18__PC', 'target', 'desmatriculado', 'PL_CURSO_ACADEMICO', 'CH_NACIONAL',
    'NU_NOTA_MEDIA_ADMISION', 'NU_NOTA_MEDIA_1_BACH__PC', 'CH_PRUEBAS_CALIFICADAS', 
    'NU_RESULTADO_ADMISION_PUNTOS', 'PL_RESOLUCION_DEFINITIVA', 'TITULACION', 'CENTROENSENANZA',
    'MINIMUMPAYMENTPAYED', 'PAID_AMOUNT', 'PAID_PERCENT', 'CH_PAGO_SUPERIOR', 
    'CH_MATRICULA_SUJETA_BECA', 'CH_AYUDA_FINANCIACION', 'CU_IMPORTE_TOTAL',
    'CH_VISITACAMPUS__PC', 'CH_ENTREVISTA_PERSONAL__PC', 'ACC_DTT_FECHAULTIMAACTIVIDAD', 
    'NU_PREFERENCIA', 'STAGENAME', 'PL_SUBETAPA',
    'CH_HIJO_EMPLEADO__PC', 'CH_HIJO_PROFESOR_ASOCIADO__PC', 'CH_HERMANOS_ESTUDIANDO_UNAV__P', 
    'CH_HIJO_MEDICO__PC', 'YEARPERSONBIRTHDATE', 'NAMEX', 'CH_FAMILIA_NUMEROSA__PC', 
    'PL_SITUACION_SOCIO_ECONOMICA', 'LEADSOURCE', 'PL_ORIGEN_DE_SOLICITUD', 
    'PL_PLAZO_ADMISION', 'RECORDTYPENAME','PLAZO_ADMISION_LIMPIO','FO_rentaFam_ges__c','CU_precioOrdinario_def__c',
    'CU_precioAplicado_def__c','PORCENTAJE_PAGADO_FINAL','tiempo_etapa_dias','tiempo_entre_etapas_dias','num_asistencias_acum', 'num_solicitudes_acum'
]


#columnas_finales = [c for c in columnas_finales if c in df_definitivo.columns]
df_definitivo = df_definitivo[columnas_seleccionadas]

# ============================================================
# 9️⃣ GUARDAR DATASET TRATAMIENTO DEFINITIVO
# ============================================================

ruta_salida = r"C:\Users\0017655\Downloads\dataset_tratamiento_final.csv"
df_definitivo.to_csv(ruta_salida, sep=";", index=False)

print(f"✅ Dataset de tratamiento definitivo guardado en: {ruta_salida}")
print(f"Dimensiones: {df_definitivo.shape}")
df_definitivo.head()


### FIN DEL SCRIPT -> meter en 01 limpieza datasets.py y comentar, generar un 02 de analisis para empezar con correlaciones, clusters, etc

Procesando 536604 filas con lógica de curso y progresión temporal...
Cruzando datos por ID18__PC y Curso Académico...
Aplicando filtro temporal progresivo...
Agrupando resultados...
Consolidando en el DataFrame maestro...
✅ Proceso completado.
✅ Dataset de tratamiento definitivo guardado en: C:\Users\0017655\Downloads\dataset_tratamiento_final.csv
Dimensiones: (536604, 48)


,ACCOUNTID,ID,ID18__PC,target,desmatriculado,PL_CURSO_ACADEMICO,CH_NACIONAL,NU_NOTA_MEDIA_ADMISION,NU_NOTA_MEDIA_1_BACH__PC,CH_PRUEBAS_CALIFICADAS,...,RECORDTYPENAME,PLAZO_ADMISION_LIMPIO,FO_rentaFam_ges__c,CU_precioOrdinario_def__c,CU_precioAplicado_def__c,PORCENTAJE_PAGADO_FINAL,tiempo_etapa_dias,tiempo_entre_etapas_dias,num_asistencias_acum,num_solicitudes_acum
0,001w000001X8jDhAAJ,0061r00000yz6vuAAA,003w000001knzGTAAY,0.0,0.0,2022/2023,True,NaN,6.0,False,...,Solicitud Admisión Grado,Diciembre,NaN,NaN,NaN,NaN,0,0,0,0
1,001w000001X8jDhAAJ,0061r00000yz6vuAAA,003w000001knzGTAAY,0.0,0.0,2022/2023,True,NaN,6.0,False,...,Solicitud Admisión Grado,Diciembre,NaN,NaN,NaN,NaN,0,0,0,0
2,001w000001X8jDhAAJ,0061r00000yz6vuAAA,003w000001knzGTAAY,0.0,0.0,2022/2023,True,NaN,6.0,False,...,Solicitud Admisión Grado,Diciembre,NaN,NaN,NaN,NaN,0,0,0,0
3,001w000001X8jDhAAJ,0061r00000yz6vuAAA,003w000001knzGTAAY,0.0,0.0,2022/2023,True,NaN,6.0,False,...,Solicitud Admisión Grado,Diciembre,NaN,NaN,NaN,NaN,0,0,0,0
4,001w000001X8jDhAAJ,0061r00000yz6vuAAA,003w000001knzGTAAY,0.0,0.0,2022/2023,True,NaN,6.0,False,...,Solicitud Admisión Grado,Diciembre,NaN,NaN,NaN,NaN,0,0,0,0


CODIGO COMPLETAMENTE UNIDO, LIMPIO Y COMENTADO

In [ ]:
# ============================================================
# SCRIPT · DATASET DE TRATAMIENTO DEFINITIVO FINAL
# ============================================================
# Objetivo:
#   - Construir el dataset final de modelización a partir de Salesforce
#   - Integrar información académica, económica, actividades y tiempos
#   - Controlar leakage de información futura
#   - Dejar el dataset listo para análisis y ML
# ============================================================

import pandas as pd
import numpy as np
from utils import crear_target, eliminar_columnas_na, calcular_tiempos_etapas, integrar_actividades_progresivo_por_curso

# Funciones auxiliares definidas en utils.py
# - crear_target: construye la variable objetivo a partir del historial de etapas
# - eliminar_columnas_na: elimina columnas con exceso de valores nulos
# - calcular_tiempos_etapas: calcula duración en cada etapa del funnel
# - integrar_actividades_progresivo_por_curso: agrega actividades acumuladas

# ============================================================
# 1️⃣ CARGA DE DATOS
# ============================================================
# Se carga el Excel completo de Salesforce
# Cada hoja corresponde a una entidad distinta
# ============================================================

ruta_excel = r"..\datos\01. Datos originales\DataSET_SF - V2.xlsx"
dfs = pd.read_excel(ruta_excel, sheet_name=None)

# Asignar cada hoja a un dataframe independiente
# El orden debe coincidir con el Excel original
oportunidad = list(dfs.values())[0]
cuenta = list(dfs.values())[1]
ecb = list(dfs.values())[2]
solicitud_ban = list(dfs.values())[3]
casos = list(dfs.values())[4]
correos = list(dfs.values())[5]
historial_actividad = list(dfs.values())[6]
historial_etapas = list(dfs.values())[7]

# ============================================================
# 2️⃣ LIMPIEZA INICIAL DE NAS Y COLUMNAS
# ============================================================
# Se eliminan columnas con un porcentaje de NA superior al umbral
# Esto reduce ruido y dimensionalidad desde el inicio
# ============================================================

def eliminar_columnas_na(df, umbral=0.9):
    """Elimina columnas con más de un umbral de valores NA"""
    return df.loc[:, df.isna().mean() < umbral]


# Limpieza genérica (no modifica los dataframes originales)
for df in [oportunidad, cuenta, ecb, solicitud_ban, casos, correos, historial_actividad, historial_etapas]:
    df = eliminar_columnas_na(df)


# Limpieza efectiva sobre los dataframes clave
oportunidad = eliminar_columnas_na(oportunidad)
cuenta = eliminar_columnas_na(cuenta)
ecb = eliminar_columnas_na(ecb)

# ============================================================
# 3️⃣ CREACIÓN DEL TARGET
# ============================================================
# Se construye la variable objetivo (target) usando el historial de etapas
# ============================================================

oportunidad = crear_target(oportunidad, historial_etapas)


# Unión de oportunidad con datos de cuenta/persona
# Se hace LEFT JOIN para no perder oportunidades

df_unido = pd.merge(
    oportunidad, 
    cuenta, 
    left_on='ACCOUNTID', 
    right_on='ID18', 
    how='left',
    suffixes=('', '_cuenta')
)


# ============================================================
# 4️⃣ CONSTRUCCIÓN VARIABLES DERIVADAS
# ============================================================
# Se crean variables explicativas a partir de campos originales
# ============================================================

# Normalización del plazo de admisión
# Se agrupan valores heterogéneos en categorías consistentes
def normalizar_plazo(x):
    if pd.isna(x): return "Rolling"
    x = str(x).strip().lower()
    if "dic" in x: return "Diciembre"
    if "mar" in x: return "Marzo"
    return "Otros"

df_unido['PLAZO_ADMISION_LIMPIO'] = df_unido['PL_PLAZO_ADMISION'].apply(normalizar_plazo)

# Unión con información económica (ECB)
# Se incorporan precios y renta familiar
ecb_vars = ['LK_oportunidad__c', 'FO_rentaFam_ges__c', 'CU_precioOrdinario_def__c', 'CU_precioAplicado_def__c']
df_definitivo = pd.merge(
    df_unido,
    ecb[ecb_vars],
    left_on='ID',
    right_on='LK_oportunidad__c',
    how='left'
)


# Cálculo del porcentaje pagado final
# Se controla división por cero
df_definitivo['PORCENTAJE_PAGADO_FINAL'] = (
    df_definitivo['CU_precioAplicado_def__c'] / df_definitivo['CU_precioOrdinario_def__c'] * 100
)
df_definitivo.loc[df_definitivo['CU_precioOrdinario_def__c'] <= 0, 'PORCENTAJE_PAGADO_FINAL'] = np.nan


# Guardado intermedio (dataset de análisis)
ruta_salida = r"..\datos\01. Datos originales\dataset_analisis_final.csv"
df_definitivo.to_csv(ruta_salida, sep=";", index=False)


# ============================================================
# 5️⃣ TIEMPO EN CADA ETAPA
# ============================================================
# Se calcula el tiempo pasado en cada etapa del funnel
# ============================================================

historial_etapas_tiempo = calcular_tiempos_etapas(historial_etapas)
df_definitivo = historial_etapas_tiempo.merge(df_definitivo, left_on='LK_Oportunidad__c', right_on='ID', how='left')

# ============================================================
# 6️⃣ HISTORIAL DE ACTIVIDADES
# ============================================================
# Se integran actividades acumuladas por curso
# Evita usar información futura respecto a la etapa
# ============================================================

df_definitivo = integrar_actividades_progresivo_por_curso(df_definitivo, historial_actividad)

# ============================================================
# 7️⃣ CONTROL DE INFORMACIÓN FUTURA (LEAKAGE)
# ============================================================
# Se eliminan variables económicas si aparecen en etapas tempranas
# ============================================================

etapas_pago = ['Solicitud', 'Pruebas', 'Admisión académica']
vars_pago = ['PAID_AMOUNT','MINIMUMPAYMENTPAYED','CU_precioAplicado_def__c','PORCENTAJE_PAGADO_FINAL']
vars_pago = [v for v in vars_pago if v in df_definitivo.columns]

mask_futuro = (df_definitivo['PL_Etapa__c'].isin(etapas_pago)) & (df_definitivo[vars_pago].notna().any(axis=1))
df_definitivo.loc[mask_futuro, vars_pago] = np.nan

# ============================================================
# 8️⃣ SELECCIÓN VARIABLES FINALES
# ============================================================
# Se define explícitamente el conjunto final de variables
# ============================================================
columnas_seleccionadas = [
    'ACCOUNTID', 'ID','ID18__PC', 'target', 'desmatriculado', 'PL_CURSO_ACADEMICO', 'CH_NACIONAL',
    'NU_NOTA_MEDIA_ADMISION', 'NU_NOTA_MEDIA_1_BACH__PC', 'CH_PRUEBAS_CALIFICADAS', 
    'NU_RESULTADO_ADMISION_PUNTOS', 'PL_RESOLUCION_DEFINITIVA', 'TITULACION', 'CENTROENSENANZA',
    'MINIMUMPAYMENTPAYED', 'PAID_AMOUNT', 'PAID_PERCENT', 'CH_PAGO_SUPERIOR', 
    'CH_MATRICULA_SUJETA_BECA', 'CH_AYUDA_FINANCIACION', 'CU_IMPORTE_TOTAL',
    'CH_VISITACAMPUS__PC', 'CH_ENTREVISTA_PERSONAL__PC', 'ACC_DTT_FECHAULTIMAACTIVIDAD', 
    'NU_PREFERENCIA', 'STAGENAME', 'PL_SUBETAPA',
    'CH_HIJO_EMPLEADO__PC', 'CH_HIJO_PROFESOR_ASOCIADO__PC', 'CH_HERMANOS_ESTUDIANDO_UNAV__P', 
    'CH_HIJO_MEDICO__PC', 'YEARPERSONBIRTHDATE', 'NAMEX', 'CH_FAMILIA_NUMEROSA__PC', 
    'PL_SITUACION_SOCIO_ECONOMICA', 'LEADSOURCE', 'PL_ORIGEN_DE_SOLICITUD', 
    'PL_PLAZO_ADMISION', 'RECORDTYPENAME','PLAZO_ADMISION_LIMPIO','FO_rentaFam_ges__c','CU_precioOrdinario_def__c',
    'CU_precioAplicado_def__c','PORCENTAJE_PAGADO_FINAL','tiempo_etapa_dias','tiempo_entre_etapas_dias','num_asistencias_acum', 'num_solicitudes_acum'
]


#columnas_finales = [c for c in columnas_finales if c in df_definitivo.columns]
df_definitivo = df_definitivo[columnas_seleccionadas]

# ============================================================
# 9️⃣ GUARDAR DATASET TRATAMIENTO DEFINITIVO
# ============================================================

ruta_salida = r"..\datos\01. Datos originales\dataset_tratamiento_final.csv"
df_definitivo.to_csv(ruta_salida, sep=";", index=False)

print(f"✅ Dataset de tratamiento definitivo guardado en: {ruta_salida}")
print(f"Dimensiones: {df_definitivo.shape}")
df_definitivo.head()

# ===============================

Hay un total de 15470 matrículas formalizadas. Un 22.03% del total de oportunidades
Hay un total de 1495 desmatriculados. Un 9.66% del total de matriculados


c:\Users\0021755\OneDrive - ViewNext\proyectoUNAV\gitUNAV\UNAV\notebooks\utils.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oportunidad['target'] = oportunidad['ID'].apply(
c:\Users\0021755\OneDrive - ViewNext\proyectoUNAV\gitUNAV\UNAV\notebooks\utils.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Procesando 536604 filas con lógica de curso y progresión temporal...
Cruzando datos por ID18__PC y Curso Académico...
Aplicando filtro temporal progresivo...
Agrupando resultados...
Consolidando en el DataFrame maestro...
✅ Proceso completado.
✅ Dataset de tratamiento definitivo guardado en: ..\datos\01. Datos originales\dataset_tratamiento_final.csv
Dimensiones: (536604, 48)


,ACCOUNTID,ID,ID18__PC,target,desmatriculado,PL_CURSO_ACADEMICO,CH_NACIONAL,NU_NOTA_MEDIA_ADMISION,NU_NOTA_MEDIA_1_BACH__PC,CH_PRUEBAS_CALIFICADAS,...,RECORDTYPENAME,PLAZO_ADMISION_LIMPIO,FO_rentaFam_ges__c,CU_precioOrdinario_def__c,CU_precioAplicado_def__c,PORCENTAJE_PAGADO_FINAL,tiempo_etapa_dias,tiempo_entre_etapas_dias,num_asistencias_acum,num_solicitudes_acum
0,001w000001X8jDhAAJ,0061r00000yz6vuAAA,003w000001knzGTAAY,0.0,0.0,2022/2023,True,NaN,6.0,False,...,Solicitud Admisión Grado,Diciembre,NaN,NaN,NaN,NaN,0,0,0,0
1,001w000001X8jDhAAJ,0061r00000yz6vuAAA,003w000001knzGTAAY,0.0,0.0,2022/2023,True,NaN,6.0,False,...,Solicitud Admisión Grado,Diciembre,NaN,NaN,NaN,NaN,0,0,0,0
2,001w000001X8jDhAAJ,0061r00000yz6vuAAA,003w000001knzGTAAY,0.0,0.0,2022/2023,True,NaN,6.0,False,...,Solicitud Admisión Grado,Diciembre,NaN,NaN,NaN,NaN,0,0,0,0
3,001w000001X8jDhAAJ,0061r00000yz6vuAAA,003w000001knzGTAAY,0.0,0.0,2022/2023,True,NaN,6.0,False,...,Solicitud Admisión Grado,Diciembre,NaN,NaN,NaN,NaN,0,0,0,0
4,001w000001X8jDhAAJ,0061r00000yz6vuAAA,003w000001knzGTAAY,0.0,0.0,2022/2023,True,NaN,6.0,False,...,Solicitud Admisión Grado,Diciembre,NaN,NaN,NaN,NaN,0,0,0,0
